<img src="images/logos/esgf2-us.png" width=250 alt="ESGF logo"></img>

# Complex Searching with `intake-esgf`

## Overview

In this tutorial we will present an interface under design to facilitate complex searching using [intake-esgf](https://github.com/esgf2-us/intake-esgf). `intake-esgf` is a small `intake` and `intake-esm` *inspired* package under development in ESGF2. Please note that there is a name collison with an existing package in PyPI and conda. You will need to install the package from [source](https://github.com/esgf2-us/intake-esgf).

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Install Package](https://github.com/esgf2-us/intake-esgf) | Necessary | |
| [Understanding of NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf.html) | Helpful | Familiarity with metadata structure |
| Familiar with [intake-esm](https://intake-esm.readthedocs.io/en/stable/) | Helpful | Similar interface |
| [Transient climate response](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2008JD010405) | Background | |
- **Time to learn**: 30 minutes

## Imports

In [1]:
from intake_esgf import ESGFCatalog

## Initializing the Catalog

As with `intake-esm` we first instantiate the catalog. However, since we will populate the catalog with search results, the catalog starts empty. Internally, we query different ESGF index nodes for information about what datasets you wish to include in your analysis. As ESGF2 is actively working on an index redesign, our catlogs by default point to a Globus (ElasticSearch) based index at ALCF (Argonne Leadership Computing Facility).

In [2]:
cat = ESGFCatalog()
print(cat)
for ind in cat.indices: # Which indices are included?
    print(ind)

Perform a search() to populate the catalog.
GlobusESGFIndex('anl-dev')


We also provide support for connecting to the ESGF1 Solr-based indices. You may specify a server or list or just include `True` to choose all the federated index nodes.

In [3]:
cat = ESGFCatalog(esgf1_indices="esgf-node.llnl.gov")  # include LLNL
cat = ESGFCatalog(esgf1_indices=["esgf-node.ornl.gov", "esgf.ceda.ac.uk"])  # ORNL & CEDA
cat = ESGFCatalog(esgf1_indices=True)  # all federated indices
for ind in cat.indices:
    print(ind)

GlobusESGFIndex('anl-dev')
SolrESGFIndex('esgf.ceda.ac.uk')
SolrESGFIndex('esgf-data.dkrz.de')
SolrESGFIndex('esgf-node.ipsl.upmc.fr')
SolrESGFIndex('esg-dn1.nsc.liu.se')
SolrESGFIndex('esgf-node.llnl.gov')
SolrESGFIndex('esgf.nci.org.au')
SolrESGFIndex('esgf-node.ornl.gov')


## Populate the catalog

Many times, an analysis will require several variables across multiple experiments. For example, if one were to compute the transient climate response (TCRE), you would need tempererature (`tas`) and carbon emissions from land (`nbp`) and ocean (`fgco2`) for a 1% CO2 increase experiment (`1pctCO2`) as well as the control experiment (`piControl`). If TCRE is not in your particular science, that is ok for this notebook. It is a motivating example and the specifics are less important than the search concepts. First, we perform a search in a familiar syntax.

In [4]:
cat.search(
    experiment_id=["piControl", "1pctCO2"],
    variable_id=["tas", "fgco2", "nbp"],
    table_id=["Amon", "Omon", "Lmon"],
)
print(cat)

   Searching indices: 100%|███████████████████████████████|8/8 [    1.85s/index]


Summary information for 399 results:
mip_era                                                     [CMIP6]
activity_id                                                  [CMIP]
institution_id    [MOHC, MRI, MPI-M, NCAR, NOAA-GFDL, NCC, NIMS-...
source_id         [UKESM1-0-LL, MRI-ESM2-0, MPI-ESM1-2-LR, CESM2...
experiment_id                                  [piControl, 1pctCO2]
member_id         [r1i1p1f2, r1i2p1f1, r1i1p1f1, r2i1p1f1, r3i1p...
table_id                                         [Lmon, Omon, Amon]
variable_id                                       [nbp, fgco2, tas]
grid_label                                            [gn, gr1, gr]
dtype: object


Internally, this launches simultaneous searches that are combined locally to provide a global view of what datasets are available. While the Solr indices themselves can be searched in distributed fashion, they will not report if an index has failed to return a response. As index nodes go down from time to time, this can leave you with a false impression that you have found all the datasets of interest. By managing the searches locally, `intake-esgf` can report back to you that an index has failed and that your results may be incomplete.

If you would like details about what `intake-esgf` is doing, look in the local cache directory (`${HOME}/.esgf/`) for a `esgf.log` file. This is a full history of everything that `intake-esgf` has searched, downloaded, or accessed. You can also look at just this session by calling `session_log()`. In this case you will see how long each index took to return a response and if any failed

In [5]:
print(cat.session_log())

2023-12-06 15:51:39 search begin experiment_id=['piControl', '1pctCO2'], variable_id=['tas', 'fgco2', 'nbp'], table_id=['Amon', 'Omon', 'Lmon']
2023-12-06 15:51:40 └─GlobusESGFIndex('anl-dev') results=329 response_time=1.33 total_time=1.34
2023-12-06 15:51:41 └─SolrESGFIndex('esgf-node.ipsl.upmc.fr') response_time=1.39 total_time=1.86
2023-12-06 15:51:41 └─SolrESGFIndex('esg-dn1.nsc.liu.se') response_time=1.68 total_time=2.28
2023-12-06 15:51:44 └─SolrESGFIndex('esgf.ceda.ac.uk') response_time=1.30 total_time=5.06
2023-12-06 15:51:45 └─SolrESGFIndex('esgf.nci.org.au') response_time=3.42 total_time=6.59
2023-12-06 15:51:46 └─SolrESGFIndex('esgf-node.ornl.gov') response_time=0.78 total_time=6.76
2023-12-06 15:51:46 └─SolrESGFIndex('esgf-data.dkrz.de') response_time=1.89 total_time=7.29
2023-12-06 15:51:54 └─SolrESGFIndex('esgf-node.llnl.gov') response_time=1.83 total_time=14.77
2023-12-06 15:51:54 search end total_time=15.26



At this stage of the search you have a catalog full of possibly relevant datasets for your analysis, stored in a `pandas` dataframe. You are free to view and manipulate this dataframe to help hone these results down. It is available to you as the `df` member of the `ESGFCatalog`. You should be careful to only remove rows as internally we could use any column in the downloading of the data. Also note that we have removed the user-facing notion of *where* the data is hosted. The `id` column of this dataframe is a list of full `dataset_ids` which includes the location information. At the point when you are ready to download data, we will choose locations automatically that are fastest for you.

In [6]:
cat.df

,mip_era,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,id
0,CMIP6,CMIP,MOHC,UKESM1-0-LL,piControl,r1i1p1f2,Lmon,nbp,gn,v20200828,[CMIP6.CMIP.MOHC.UKESM1-0-LL.piControl.r1i1p1f...
1,CMIP6,CMIP,MRI,MRI-ESM2-0,1pctCO2,r1i2p1f1,Omon,fgco2,gn,v20210311,[CMIP6.CMIP.MRI.MRI-ESM2-0.1pctCO2.r1i2p1f1.Om...
2,CMIP6,CMIP,MPI-M,MPI-ESM1-2-LR,piControl,r1i1p1f1,Omon,fgco2,gn,v20190710,[CMIP6.CMIP.MPI-M.MPI-ESM1-2-LR.piControl.r1i1...
3,CMIP6,CMIP,NCAR,CESM2-FV2,piControl,r1i1p1f1,Amon,tas,gn,v20191120,[CMIP6.CMIP.NCAR.CESM2-FV2.piControl.r1i1p1f1....
4,CMIP6,CMIP,NOAA-GFDL,GFDL-CM4,1pctCO2,r1i1p1f1,Amon,tas,gr1,v20180701,[CMIP6.CMIP.NOAA-GFDL.GFDL-CM4.1pctCO2.r1i1p1f...
...,...,...,...,...,...,...,...,...,...,...,...
1304,CMIP6,CMIP,NASA-GISS,GISS-E2-1-G,1pctCO2,r102i1p1f1,Lmon,nbp,gn,v20190815,[CMIP6.CMIP.NASA-GISS.GISS-E2-1-G.1pctCO2.r102...
1309,CMIP6,CMIP,MRI,MRI-ESM2-0,1pctCO2,r1i2p1f1,Amon,tas,gn,v20191205,[CMIP6.CMIP.MRI.MRI-ESM2-0.1pctCO2.r1i2p1f1.Am...
2048,CMIP6,CMIP,MIROC,MIROC-ES2H,piControl,r1i1p4f2,Omon,fgco2,gr1,v20230904,[CMIP6.CMIP.MIROC.MIROC-ES2H.piControl.r1i1p4f...
2050,CMIP6,CMIP,E3SM-Project,E3SM-2-0-NARRM,1pctCO2,r1i1p1f1,Amon,tas,gr,v20230427,[CMIP6.CMIP.E3SM-Project.E3SM-2-0-NARRM.1pctCO...


## Model Groups

However, `intake-esgf` also provides you with some tools to help locate relevant data for your analysis. When conducting these kinds of analyses, we are seeking for unique combinations of a `source_id`, `member_id`, and `grid_label` that have all the variables that we need. We call these *model groups*. In an ESGF search, it is common to find a model that has, for example, a `tas` for `r1i1p1f1` but not a `fgco2`. Sorting this out is time consuming and labor intensive. So first, we provide you a function to print out all model groups with the following function.

In [ ]:
cat.model_groups().to_frame()

The function `model_groups()` returns a pandas Series (converted to a dataframe here for printing) with all unique combinations of (`source_id`,`member_id`,`grid_label`) along with the dataset count for each. This helps illustrate why it can be so difficult to locate all the data relevant to a given analysis. At the time of this writing, there are 148 model groups but relatively few of them with all 6 (2 experiments and 3 variables) datasets that we need. Furthermore, you cannot rely on a model group using `r1i1p1f1` for its primary result. The results above show that UKESM does not even use `f1` at all, further complicating the process of finding results.

In addition to this notion of *model groups*, `intake-esgf` provides you a method `remove_incomplete()` for determing which model groups you wish to keep in the current search. Internally, we will group the search results dataframe by model groups and apply a function of your design to the grouped portion of the dataframe. For example, for the current work, I could just check that there are 6 datasets in the sub-dataframe.

In [ ]:
def shall_i_keep_it(sub_df):
    if len(sub_df) == 6:
        return True
    return False


cat.remove_incomplete(shall_i_keep_it)
cat.model_groups().to_frame()

You could write a much more complex check--it depends on what is relevant to your analysis. The effect is that the list of possible models with consistent results is now much more manageable. This method has the added benefit of forcing the user to be concrete about which models were included in an analysis.

## Removing Additional Variants

It may also be that you wish to only include a single `member_id` in your analysis. The above search shows we have a few models with multiple variants that have all 6 required datasets. To be fair to those that only have 1, you may wish to only keep the *smallest* variant. We also provide this function as part of the `ESGFCatalog` object.


In [ ]:
cat.remove_ensembles()
cat.model_groups().to_frame()

## Summary

At this point, you would be ready to use `to_dataset_dict()` to download and load all datasets into a dictionary for analysis. The point of this notebook however is to expose the search capabilities. It is our goal to make annoying and time-consuming tasks easier by providing you smart interfaces for common operations. Let us [know](https://github.com/esgf2-us/intake-esgf/issues) what else is painful for you in locating relevant data for your science.